In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import findspark
findspark.init()

import pyspark
from pyspark.sql import *
import pyspark.sql.functions as func
from pyspark.sql.types import *

# Language processing
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer

# Language processing with TextBlob
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer

from collections import Counter

import sys
sys.path.append('../src')

from plot_correlation import plot_corr_mats, plot_metrics_distrib, plot_nlp_daily_metrics, plot_nlp_daily_distrib, plot_nlp_sample_distrib

%load_ext autoreload
%autoreload 2

sns.set()

In [ ]:
# Create spark session
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
def spark_to_pandas(spark_metrics):
    metrics_pd = spark_metrics.toPandas()
    metrics_pd = metrics_pd.set_index('creation_date')
    metrics_pd = metrics_pd.sort_index()
    return metrics_pd

In [ ]:
def get_metrics(filename, sql_query, sql_table_name):
    spark_metrics = spark.read.load(filename)
    spark_metrics.registerTempTable(sql_table_name)
    avg_metrics = spark.sql(sql_query)

    pd_metrics = spark_to_pandas(spark_metrics)
    pd_metrics_avg = spark_to_pandas(avg_metrics)
    pd_metrics_n = pd_metrics.div(pd_metrics['msg_count'], axis=0)
    pd_metrics_avg_n = pd_metrics_avg.div(pd_metrics_avg['msg_count_60d_avg'], axis=0)
    
    return pd_metrics_n, pd_metrics_avg_n, pd_metrics, pd_metrics_avg

### NLTK Metrics

In [ ]:
nltk_full_name = '../data/nlp_nltk_metrics_daily_full_0.01.parquet/'
nltk_15_17_name = '../data/nlp_nltk_metrics_daily_15_17_0.1.parquet/'

nltk_sql_table_name = 'nltk_metrics'
nltk_sql_query = (f"""
SELECT
    creation_date,
    
    AVG(msg_count) OVER (
        ORDER BY creation_date
        RANGE BETWEEN 30 PRECEDING AND 30 FOLLOWING
    ) AS msg_count_60d_avg,

    AVG(sum_nltk_negativity) OVER (
        ORDER BY creation_date
        RANGE BETWEEN 30 PRECEDING AND 30 FOLLOWING
    ) AS nltk_negativity_60d_avg,

    AVG(sum_nltk_neutrality) OVER (
        ORDER BY creation_date
        RANGE BETWEEN 30 PRECEDING AND 30 FOLLOWING
    ) AS nltk_neutrality_60d_avg,
    
    AVG(sum_nltk_positivity) OVER (
        ORDER BY creation_date
        RANGE BETWEEN 30 PRECEDING AND 30 FOLLOWING
    ) AS nltk_positivity_60d_avg

FROM {nltk_sql_table_name}
""")

def plot_nltk_metrics(filename, sql_query, sql_table_name):
    m_n, m_avg_n, m, m_avg = get_metrics(filename, sql_query, sql_table_name)
    
    def plot_averages(neg=True, neu=True, pos=True):
        plt.figure(figsize=(20, 10))
        title = None
        labels = []
        if neg:
            title = 'Negativity avg'
            labels.append('Negativity avg')
            plt.plot(m_avg_n['nltk_negativity_60d_avg'])
        if neu:
            title = 'Neutrality avg' if title is None else title + ' VS Neutrality avg'
            labels.append('Neutrality avg')
            plt.plot(m_avg_n['nltk_neutrality_60d_avg'])
        if pos:
            title = 'Positivity avg' if title is None else title + ' VS Positivity avg'
            labels.append('Positivity avg')
            plt.plot(m_avg_n['nltk_positivity_60d_avg'])
        plt.title(title + ' scores over time')
        plt.xlabel('Date')
        plt.ylabel('Score')
        plt.legend(labels)
    
    def plot_stat(neg=False, neu=False, pos=False):
        if neg or neu or pos:
            plt.figure(figsize=(20, 10))
        if neg:
            plt.plot(m_n['sum_nltk_negativity'])
            plt.plot(m_avg_n['nltk_negativity_60d_avg'])
            plt.title('Negativity scores over time')
            plt.legend(['Negativity', 'Negativity avg'])
        elif neu:
            plt.plot(m_n['sum_nltk_neutrality'])
            plt.plot(m_avg_n['nltk_neutrality_60d_avg'])
            plt.title('Neutrality scores over time')
            plt.legend(['Neutrality', 'Neutrality avg'])
        elif pos:
            plt.plot(m_n['sum_nltk_positivity'])
            plt.plot(m_avg_n['nltk_positivity_60d_avg'])
            plt.title('Positivity scores over time')
            plt.legend(['Positivity', 'Positivity avg'])
        else:
            print('Please set one of the following parameter to True: [neg, neu, pos]')
            return
        
        plt.xlabel('Date')
        plt.ylabel('Score')
            
    return plot_averages, plot_stat

#### NLTK 2005 - 2017

In [ ]:
nltk_plot_averages, nltk_plot_stat = plot_nltk_metrics(nltk_full_name, nltk_sql_query, nltk_sql_table_name)

In [ ]:
nltk_plot_averages()

In [ ]:
nltk_plot_averages(neu=False)

In [ ]:
nltk_plot_averages(neg=False, pos=False)

In [ ]:
nltk_plot_stat(neg=True)

In [ ]:
nltk_plot_stat(neu=True)

In [ ]:
nltk_plot_stat(pos=True)

#### NLTK 2015 - 2017

In [ ]:
nltk_plot_averages, nltk_plot_stat = plot_nltk_metrics(nltk_15_17_name, nltk_sql_query, nltk_sql_table_name)

In [ ]:
nltk_plot_averages()

In [ ]:
nltk_plot_averages(neu=False)

In [ ]:
nltk_plot_averages(neg=False, pos=False)

In [ ]:
nltk_plot_stat(neg=True)

In [ ]:
nltk_plot_stat(neu=True)

In [ ]:
nltk_plot_stat(pos=True)

### TextBlob Metrics

In [ ]:
blob_full_name = '../data/nlp_blob_metrics_daily_full_0.01.parquet/'
blob_15_17_name = '../data/nlp_blob_metrics_daily_15_17_0.1.parquet/'

blob_sql_table_name = 'blob_metrics'
blob_sql_query = (f"""
SELECT
    creation_date,
    
    AVG(msg_count) OVER (
        ORDER BY creation_date
        RANGE BETWEEN 30 PRECEDING AND 30 FOLLOWING
    ) AS msg_count_60d_avg,

    AVG(sum_blob_polarity) OVER (
        ORDER BY creation_date
        RANGE BETWEEN 30 PRECEDING AND 30 FOLLOWING
    ) AS text_blob_polarity_60d_avg,

    AVG(sum_blob_subjectivity) OVER (
        ORDER BY creation_date
        RANGE BETWEEN 30 PRECEDING AND 30 FOLLOWING
    ) AS text_blob_subjectivity_60d_avg

FROM {blob_sql_table_name}
""")

def plot_blob_metrics(filename, sql_query, sql_table_name):
    m_n, m_avg_n, m, m_avg = get_metrics(filename, sql_query, sql_table_name)
    
    def plot_stat(pol=False, pol_avg=False, subj=False, subj_avg=False):
        if pol_avg or pol or subj_avg or subj:
            plt.figure(figsize=(20, 10))
            title = None
            labels = []
            if pol:
                plt.plot(m_n['sum_blob_polarity'])
                title = 'Polarity'
                labels.append('Polarity')
            if pol_avg:
                plt.plot(m_avg_n['text_blob_polarity_60d_avg'])
                title = 'Polarity avg' if title is None else title + ' VS Polarity avg'
                labels.append('Polarity avg')
            if subj:
                plt.plot(m_n['sum_blob_subjectivity'])
                title = 'Subjectivity' if title is None else title + ' VS Subjectivity'
                labels.append('Subjectivity')
            if subj_avg:
                plt.plot(m_avg_n['text_blob_subjectivity_60d_avg'])
                title = 'Subjectivity avg' if title is None else title + ' VS Subjectivity avg'
                labels.append('Subjectivity avg')
            plt.title(title + ' scores over time')
            plt.xlabel('Date')
            plt.ylabel('Score')
            plt.legend(labels)
        else:
            print('Please set one of the following parameter to True: [pol, pol_avg, subj, subj_avg]')
            
    return plot_stat

#### Blob 2005 - 2017

In [ ]:
blob_plot_stat = plot_blob_metrics(blob_full_name, blob_sql_query, blob_sql_table_name)

In [ ]:
blob_plot_stat(pol=True, pol_avg=True)

In [ ]:
blob_plot_stat(pol_avg=True)

In [ ]:
blob_plot_stat(subj=True, subj_avg=True)

In [ ]:
blob_plot_stat(subj_avg=True)

#### Blob 2015-2017

In [ ]:
blob_plot_stat = plot_blob_metrics(blob_15_17_name, blob_sql_query, blob_sql_table_name)

In [ ]:
blob_plot_stat(pol=True, pol_avg=True)

In [ ]:
blob_plot_stat(subj=True, subj_avg=True)

In [ ]:
blob_plot_stat(pol_avg=True)

In [ ]:
blob_plot_stat(subj_avg=True)

### Bad words metrics

In [ ]:
bw_full_name = '../data/nlp_bw_metrics_daily_full_0.01.parquet/'
bw_15_17_name = '../data/nlp_bw_metrics_daily_15_17_0.1.parquet/'

bw_sql_table_name = 'bw_metrics'
bw_sql_query = (f"""
SELECT
    creation_date,
    
    AVG(msg_count) OVER (
        ORDER BY creation_date
        RANGE BETWEEN 30 PRECEDING AND 30 FOLLOWING
    ) AS msg_count_60d_avg,

    AVG(sum_nb_bw_matches) OVER (
        ORDER BY creation_date
        RANGE BETWEEN 30 PRECEDING AND 30 FOLLOWING
    ) AS nb_bw_matches_60d_avg

FROM {bw_sql_table_name}
""")

def plot_bw_metrics(filename, sql_query, sql_table_name):
    m_n, m_avg_n, m, m_avg = get_metrics(filename, sql_query, sql_table_name)
    
    def plot_stat(bw=False, bw_avg=False):
        if bw or bw_avg:
            plt.figure(figsize=(20, 10))
            title = None
            labels = []
            if bw:
                plt.plot(m_n['sum_nb_bw_matches'])
                title = 'Bad words'
                labels.append('Bad words')
            if bw_avg:
                plt.plot(m_avg_n['nb_bw_matches_60d_avg'])
                title = 'Bad words avg' if title is None else title + ' VS Bad words avg'
                labels.append('Bad words avg')
                
            plt.title(title + ' scores over time')
            plt.xlabel('Date')
            plt.ylabel('Score')
            plt.legend(labels)
        else:
            print('Please set one of the following parameter to True: [bw, bw_avg]')
            
    return plot_stat

#### Bad words 2005-2017

In [ ]:
bw_plot_stat = plot_bw_metrics(bw_full_name, bw_sql_query, bw_sql_table_name)

In [ ]:
bw_plot_stat(bw=True, bw_avg=True)

In [ ]:
bw_plot_stat(bw_avg=True)

#### Bad words 2015-2017

In [ ]:
bw_plot_stat = plot_bw_metrics(bw_15_17_name, bw_sql_query, bw_sql_table_name)

In [ ]:
bw_plot_stat(bw=True, bw_avg=True)

In [ ]:
bw_plot_stat(bw_avg=True)

### Hate words metrics

In [ ]:
hw_full_name = '../data/nlp_hw_metrics_daily_full_0.01.parquet/'
hw_15_17_name = '../data/nlp_hw_metrics_daily_15_17_0.1.parquet/'

hw_sql_table_name = 'hw_metrics'
hw_sql_query = (f"""
SELECT
    creation_date,
    
    AVG(msg_count) OVER (
        ORDER BY creation_date
        RANGE BETWEEN 30 PRECEDING AND 30 FOLLOWING
    ) AS msg_count_60d_avg,

    AVG(sum_nb_hw_matches) OVER (
        ORDER BY creation_date
        RANGE BETWEEN 30 PRECEDING AND 30 FOLLOWING
    ) AS nb_hw_matches_60d_avg

FROM {hw_sql_table_name}
""")

def plot_hw_metrics(filename, sql_query, sql_table_name):
    m_n, m_avg_n, m, m_avg = get_metrics(filename, sql_query, sql_table_name)
    
    def plot_stat(hw=False, hw_avg=False):
        if hw or hw_avg:
            plt.figure(figsize=(20, 10))
            title = None
            labels = []
            if hw:
                plt.plot(m_n['sum_nb_hw_matches'])
                title = 'Hate words'
                labels.append('Hate words')
            if hw_avg:
                plt.plot(m_avg_n['nb_hw_matches_60d_avg'])
                title = 'Hate words avg' if title is None else title + ' VS Hate words avg'
                labels.append('Hate words avg')
            plt.title(title + ' scores over time')
            plt.xlabel('Date')
            plt.ylabel('Score')
            plt.legend(labels)
        else:
            print('Please set one of the following parameter to True: [hw, hw_avg]')
            
    return plot_stat

#### Hate words 2005-2017

In [ ]:
hw_plot_stat = plot_hw_metrics(hw_full_name, hw_sql_query, hw_sql_table_name)

In [ ]:
hw_plot_stat(hw=True, hw_avg=True)

In [ ]:
hw_plot_stat(hw_avg=True)

#### Hate words 2015-2017

In [ ]:
hw_plot_stat = plot_hw_metrics(hw_15_17_name, hw_sql_query, hw_sql_table_name)

In [ ]:
hw_plot_stat(hw=True, hw_avg=True)

In [ ]:
hw_plot_stat(hw_avg=True)

### Refined Hate Words

In [ ]:
ref_hw_full_name = '../data/nlp_ref_hw_metrics_daily_full_0.01.parquet/'
ref_hw_15_17_name = '../data/nlp_ref_hw_metrics_daily_15_17_0.1.parquet/'

ref_hw_sql_table_name = 'ref_hw_metrics'
ref_hw_sql_query = (f"""
SELECT
    creation_date,
    
    AVG(msg_count) OVER (
        ORDER BY creation_date
        RANGE BETWEEN 30 PRECEDING AND 30 FOLLOWING
    ) AS msg_count_60d_avg,
    
    AVG(sum_hw_ref_intensity) OVER (
        ORDER BY creation_date
        RANGE BETWEEN 30 PRECEDING AND 30 FOLLOWING
    ) AS hw_ref_intensity_60d_avg,

    AVG(sum_nb_hw_ref_matches) OVER (
        ORDER BY creation_date
        RANGE BETWEEN 30 PRECEDING AND 30 FOLLOWING
    ) AS nb_hw_ref_matches_60d_avg

FROM {ref_hw_sql_table_name}
""")

def plot_ref_hw_metrics(filename, sql_query, sql_table_name):
    m_n, m_avg_n, m, m_avg = get_metrics(filename, sql_query, sql_table_name)
    
    def plot_stat(ref_hw=False, ref_hw_avg=False, intensity=False, intensity_avg=False):
        if ref_hw or ref_hw_avg or intensity or intensity_avg:
            plt.figure(figsize=(20, 10))
            title = None
            labels = []
            if ref_hw:
                plt.plot(m_n['sum_nb_hw_ref_matches'])
                title = 'Hate words refined'
                labels.append('Hate words refined')
            if ref_hw_avg:
                plt.plot(m_avg_n['nb_hw_ref_matches_60d_avg'])
                title = 'Hate words refined avg' if title is None else title + ' VS Hate words refined avg'
                labels.append('Hate words refined avg')
            if intensity:
                plt.plot(m_n['sum_hw_ref_intensity'])
                title = 'Hate words refined intensity' if title is None else title + ' VS Hate words refined intensity'
                labels.append('Hate words refined intensity')
            if intensity_avg:
                plt.plot(m_avg_n['hw_ref_intensity_60d_avg'])
                title = 'Hate words refined intensity avg' if title is None else title + ' VS Hate words refined intensity avg'
                labels.append('Hate words refined intensity avg')
            
            plt.title(title + ' scores over time')
            plt.xlabel('Date')
            plt.ylabel('Score')
            plt.legend(labels)
        else:
            print('Please set one of the following parameter to True: [ref_hw, ref_hw_avg, intensity, intensity_avg]')
            
    return plot_stat

In [ ]:
ref_hw_plot_stat = plot_ref_hw_metrics(ref_hw_full_name, ref_hw_sql_query, ref_hw_sql_table_name)

In [ ]:
ref_hw_plot_stat(ref_hw_avg=True, intensity_avg=True)

In [ ]:
ref_hw_plot_stat(ref_hw=True, ref_hw_avg=True)

In [ ]:
ref_hw_plot_stat(ref_hw_avg=True)

In [ ]:
ref_hw_plot_stat(intensity=True, intensity_avg=True)

In [ ]:
ref_hw_plot_stat(intensity_avg=True)

### Correlation analysis

In [ ]:
nltk_n, nltk_avg_n, _, _ = get_metrics(nltk_full_name, nltk_sql_query, nltk_sql_table_name)
blob_n, blob_avg_n, _, _ = get_metrics(blob_full_name, blob_sql_query, blob_sql_table_name)
bw_n, bw_avg_n, _, _ = get_metrics(bw_full_name, bw_sql_query, bw_sql_table_name)
hw_n, hw_avg_n, _, _ = get_metrics(hw_full_name, hw_sql_query, hw_sql_table_name)
hw_ref_n, hw_ref_avg_n, _, _ = get_metrics(ref_hw_full_name, ref_hw_sql_query, ref_hw_sql_table_name)
nlp_n = pd.concat([nltk_n, blob_n, bw_n, hw_n, hw_ref_n], axis=1, sort=True).drop('msg_count', axis=1)

In [ ]:
nlp_n.columns = ['neg', 'neu', 'pos', 'pol', 'subj', 'bw', 'hw', 'hw_ref', 'intensity']
pd.to_pickle(nlp_n, '../data/daily_nlp_metrics_merged.pkl')

In [ ]:
nlp_n = pd.read_pickle('../data/daily_nlp_metrics_merged.pkl').dropna()
# Remove outliers assuming gaussian distribution
nlp_n_cleaned = nlp_n[(sp.stats.zscore(nlp_n) < 3).all(axis=1)]

In [ ]:
plot_nlp_daily_distrib([nlp_n, nlp_n_cleaned], ['NLP', 'NLP cleaned'])

In [ ]:
nlp_n.corr()

In [ ]:
plot_corr_mats([nlp_n, nlp_n_cleaned], 'NLP metrics', ['pearson', 'spearman'], ['', ' cleaned'])

In [ ]:
plot_nlp_daily_metrics([nlp_n, nlp_n_cleaned], ['NLP', 'NLP cleaned'])

### Sample full messages plots

In [ ]:
nlp_full = spark.read.load('../data/subreddit_nlp_full_0.001.parquet/')

In [ ]:
nlp_sample = nlp_full.sample(withReplacement=False, fraction=0.4, seed=0)

In [ ]:
pd_nlp_sample = nlp_sample.toPandas()

In [ ]:
pd_sample = pd_nlp_sample.drop(['id', 'subreddit', 'subreddit_id', 'creation_date', 'body', 'hw_ref_intensity'], axis=1)
pd_sample.columns = ['neg', 'neu', 'pos', 'pol', 'subj', 'bw', 'hw', 'hw_ref']
pd_sample.head(5)

In [ ]:
plot_nlp_sample_distrib([pd_sample], ['NLP sample'])